### Jacobian Matrix
#### FOM vs. ROM: Stiffness and Condition Number

Using linear multistep formulas of the type

\begin{equation}
\mathbf{y}_n = \sum_{j=1}^{K_1} \alpha_j \mathbf{y}_{n-j} + h_n \sum_{j=0}^{K_2} \beta_j \mathbf{f}\left(t_{n-j}, \mathbf{y}_{n-j}\right),
\end{equation}

the choice $K_1 = q$, $ K_2 = 0 $ results in the backward differentiation formula (BDF) method of order $ q $:

\begin{equation}
\mathbf{y}_n = \sum_{j=1}^q \alpha_j \mathbf{y}_{n-j} + h_n \beta_0 \mathbf{f}\left(t_n, \mathbf{y}_n\right).
\end{equation}

To solve this implicit equation, the Newton–Raphson (NR) iteration technique is employed. At iteration $ m $ of time step $ n $, the following linear system must be solved:

\begin{align}
\mathbf{P}\left(\mathbf{y}_n^{[m+1]} - \mathbf{y}_n^{[m]}\right)
&= -\mathbf{r}\left(\mathbf{y}_n^{[m]}\right) \\
&= \sum_{j=1}^q \alpha_j \mathbf{y}_{n-j} + h_n \beta_0 \mathbf{f}\left(\mathbf{y}_n^{[m]}\right) - \mathbf{y}_n^{[m]}.
\end{align}

Here, the matrix $ \mathbf{P} $ is the Jacobian of the residual $ \mathbf{r} $ with respect to $ \mathbf{y} $, and is given by

\begin{equation}
\mathbf{P} = \frac{\partial \mathbf{r}}{\partial \mathbf{y}} = \mathbf{I} - h_n \beta_0 \mathbf{J},
\end{equation}

where $ \mathbf{J} $ is the Jacobian of the function $ \mathbf{f} $ with respect to $ \mathbf{y} $.

In the simplest case, corresponding to a first-order BDF method (also known as the backward Euler method), we have $ \beta_0 = 1 $. An important metric in analyzing the difficulty of solving the linear system is the condition number of the matrix $ \mathbf{P} $. This gives insight into how ill-conditioned the system is, and therefore how sensitive the solution is to numerical errors.

The condition number $ \kappa(\mathbf{P}) $ is given by the ratio of the largest to smallest eigenvalues in magnitude:

\begin{equation}
\kappa(\mathbf{P}) = \frac{\lambda_{\max}}{\lambda_{\min}},
\end{equation}

where the eigenvalues are computed for the time-dependent matrix $ \mathbf{P}(t, \mathbf{y}) $, defined as

\begin{equation}
\mathbf{P}(t, \mathbf{y}) = \mathbf{I} - t \mathbf{J}(t, \mathbf{y}).
\end{equation}

In [ ]:
import os
import sys
import numpy as np
import scipy as sp
import joblib as jl
import dill as pickle

from tqdm import tqdm
from typing import *

In [ ]:
import matplotlib.pyplot as plt
style = "/home/zanardi/Workspace/Research/styles/matplotlib/paper_1column.mplstyle"
plt.style.use(style)

In [ ]:
sys.path.append("/home/zanardi/Codes/ML/ROMAr/romar/")
from romar import env
from romar import utils
from romar.systems import BoxAd
from romar import postproc as pp

Set enviroment

In [ ]:
env_opts = {
  "backend": "numpy",
  "device": "cpu",
  "device_idx": 0,
  "nb_threads": 2,
  "epsilon": None,
  "floatx": "float64",
  "seed": 0
}
env.set(**env_opts)

Set inputs

In [ ]:
# Trajectories indices
irange = [0, 100]
# Parallel workers
nb_workers = 10
# Paths
prefix = "/home/zanardi/Codes/ML/ROMAr/runs/run_04_cc/"
paths = {
  # > ROM basis
  "basis": prefix + "/max_mom_2/models/cobras/basis_varimax_psi.p",
  # > Path to solutions folder
  "data": prefix + "/data/rho_fixed/test/",
  # > Thermochemical database
  "dtb": "/home/zanardi/Codes/ML/ROMAr/romar/examples/database/",
  # > Output folder
  "out": "./jacobian/"
}
# Time limit
tlim = [1e-9, 1e-3]
# Number of ROM dimensions
rdims = np.arange(7,11)

In [ ]:
os.makedirs(paths["out"], exist_ok=True)

Initialize 0D thermochemical system

In [ ]:
system = BoxAd(
  species={k: paths["dtb"] + "/species/" + k + ".json" for k in ("Ar", "Arp", "em")},
  kin_dtb=paths["dtb"] + "/rates/kin_fit.p",
  rad_dtb=paths["dtb"] + "/rates/rad_fit.p",
  use_rad=True,
  use_proj=False,
  use_tables=False
)

Evaluate FOM/ROM Jacobians

In [ ]:
def evaluate_parallel(
  irange,
  nb_workers,
  eval_fun,
  **kwargs
):
  iterable = tqdm(
    iterable=range(*irange),
    ncols=80,
    desc="  Cases",
    file=sys.stdout
  )
  return jl.Parallel(nb_workers)(
    jl.delayed(
      env.make_fun_parallel(eval_fun)
    )(index=i, **kwargs) for i in iterable
  )

def evaluate_fom(
  system,
  path,
  index,
  tlim=None
):
  system.use_rom = False
  # Load test case
  icase = utils.load_case(path=path, index=index)
  t, rho, y = [icase[k] for k in ("t", "rho", "y")]
  # Time window
  if (tlim is not None):
    i = (t >= np.amin(tlim)) * (t <= np.amax(tlim))
    t = t[i]
    y = y[:,i]
  # Compute timescale and condition number
  tmin, knum = _compute_tmin_knum(system, t, y, rho)
  return {
    "t": t,
    "tmin": tmin,
    "knum": knum,
  }

def evaluate_rom(
  system,
  path,
  index,
  tout=0.0,
  tlim=None
):
  system.use_rom = True
  # Load test case
  icase = utils.load_case(path=path, index=index)
  t, y0, rho = [icase[k] for k in ("t", "y0", "rho")]
  # Time window
  if (tlim is not None):
    i = (t >= np.amin(tlim)) * (t <= np.amax(tlim))
    t = t[i]
  # Solve ROM
  z, _ = system.solve_rom(t, y0, rho, tout=tout, decode=False)
  # Postprocess
  if ((z is not None) and (z.shape[1] == len(t))):
    # > Compute energy
    zhat = z[:system.rom.size_zhat]
    energy = np.sum(zhat*zhat, axis=0)
    # > Compute timescale and condition number
    tmin, knum = _compute_tmin_knum(system, t, z, rho)
    # > Return data
    return {
      "t": t,
      "tmin": tmin,
      "knum": knum,
      "energy": energy
    }

def build_rom(system, path_to_basis, rdim):
  with open(path_to_basis, "rb") as file:
    basis = pickle.load(file)
  system.rom.build(
    phi=basis["phi"][rdim],
    psi=basis["psi"][rdim],
    **{k: basis[k] for k in ("mask", "xref", "xscale")}
  )

def _compute_tmin_knum(
  system,
  t,
  y,
  rho
):
  tmin, knum = [], []
  for (i, yi) in enumerate(y.T):
    # Compute the Jacobian
    yi = system.set_up(yi, rho)
    Ji = system.jac(t[i], yi)
    li = sp.linalg.eigvals(Ji)
    # Compute the smallest timescale
    itmin = 1.0/np.abs(li.real).max()
    tmin.append(itmin)
    # Compute the condition number
    li = np.abs(1.0 - t[i] * li.real)
    iknum = li.max() / li.min()
    knum.append(iknum)
  tmin = np.asarray(tmin)
  knum = np.asarray(knum)
  return tmin, knum

In [ ]:
data = {}
kwargs = dict(
  irange=irange,
  nb_workers=nb_workers,
  system=system,
  path=paths["data"],
  tlim=tlim
)
# Evaluate FOM
data["FOM"] = evaluate_parallel(
  eval_fun=evaluate_fom,
  **kwargs
)
# Evaluate ROMs
for rdim in rdims:
  build_rom(system, paths["basis"], rdim)
  data[fr"$r={rdim}$"] = evaluate_parallel(
    eval_fun=evaluate_rom,
    **kwargs
  )

In [ ]:
stats = {}
for (model, trajs) in data.items():
  stats[model] = {"t": trajs[0]["t"]}
  for k in trajs[0].keys():
    if (k == "t"):
      continue
    kdata = np.vstack([traj[k] for traj in trajs])
    stats[model][k] = {
      "size": len(kdata),
      "mean": np.mean(kdata, axis=0),
      "sem": sp.stats.sem(kdata, axis=0)
    }

Plot Jacobian-related quantities

In [ ]:
for (k, ylabel) in (
  ("knum",   r"$\kappa$"),
  ("tmin", r"$\tau$ [s]")
):
  pp.plot_err_ci_evolution(
    x=stats["FOM"]["t"],
    mean={m: stats[m][k]["mean"] for m  in stats.keys()},
    sem={m: stats[m][k]["sem"] for m  in stats.keys()},
    size=stats["FOM"][k]["size"],
    alpha=1.0,
    xlim=tlim,
    ylim=None,
    labels=[r"$t$ [s]", ylabel],
    scales=["log", "log"],
    legend_loc="best",
    figname=paths["out"] + f"/{k}.png",
    save=True,
    show=False
  )